In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV
from collections import OrderedDict
from time import time

import utils
import plotter

from pydotplus import graph_from_dot_data
from IPython.display import Image
from sklearn.tree import export_graphviz

%matplotlib inline
%config InlineBackend.figure_format='retina'

from sklearn.model_selection import train_test_split

PATH = os.getcwd()
RNG_SEED = 42
np.random.seed(seed=RNG_SEED)

In [ ]:
data_path_jarvis = os.path.join(PATH, './data/descriptors/jarvis.bin')
data_path_magpie = os.path.join(PATH, './data/descriptors/magpie.bin')
data_path_deltasoap_212 = os.path.join(PATH, './data/descriptors/df_deltasoap_212.pkl')
data_path_soap_reduced = os.path.join(PATH, './data/descriptors/df_soap_reduced.pkl')
data_path_soap_reduced_host = os.path.join(PATH, './data/descriptors/df_soap_reduced_host.pkl')
data_path_soap_reduced_delta = os.path.join(PATH, './data/descriptors/df_soap_reduced_delta.pkl')

data_path = [data_path_jarvis, 
             data_path_magpie, 
             data_path_deltasoap_212, 
             data_path_soap_reduced, 
             data_path_soap_reduced_host, 
             data_path_soap_reduced_delta]
x_value_raw = {}
x_label = ['jarvis', 
           'magpie',
           'deltasoap_212',
           'soap_reduced',
           'soap_reduced_host',
           'soap_reduced_delta']

for path, label in zip(data_path, x_label):
    with open(path, 'rb') as f:
        x_value_raw[label] = pickle.load(f)

In [ ]:
X_merge = pd.concat([x_value_raw['jarvis'][0], x_value_raw['magpie'][0]], axis=1)

In [ ]:
corr = X_merge.corr().abs()
X_data = utils.corr_reduction(corr, 0.8, X_merge)

In [ ]:
X_imp_based_delta = pd.concat([X_data, x_value_raw['deltasoap_212'].set_index(X_data.index)], axis=1)
X_soap_reduced = pd.concat([X_data, x_value_raw['soap_reduced'].set_index(X_data.index)], axis=1)
X_soap_reduced_delta = pd.concat([X_data, x_value_raw['soap_reduced_delta'].set_index(X_data.index)], axis=1)
y = x_value_raw['jarvis'][1]

In [ ]:
scheme_label = {'X_imp_based_delta': X_imp_based_delta, 
                'X_soap_reduced_delta': X_soap_reduced_delta}
classic_models = OrderedDict()
y = x_value_raw['jarvis'][1]

for lab, label in tqdm(scheme_label.items()):
    models = [RandomForestRegressor(), GradientBoostingRegressor(learning_rate=0.1,
                                                                max_depth=3,
                                                                n_estimators=1500,
                                                                subsample=0.7)]
    for model in models:
        X = label
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=RNG_SEED)
        model, result_dict = fit_evaluate_model(model, X_train, y_train, X_test, y_test, lab)
        df_classics = append_result_df(df_classics, result_dict)
        classic_models = append_model_dict(classic_models, model, label)

In [ ]:
#permutation importance
from sklearn.inspection import permutation_importance
X = X_imp_based_delta
y = x_value_raw['jarvis'][1]
rf = df_classics['model'].iloc[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=RNG_SEED)
result = permutation_importance(
    rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
)

sorted_importances_idx = result.importances_mean.argsort()[:10]
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

In [ ]:
#under/over estimation
y_test_data = df_classics['y_test'].iloc[1]
y_predict_data = df_classics['predict'].iloc[1]
difference = y_predict_data - y_test_data
dd = pd.DataFrame(difference).reset_index().drop(axis = 1, labels = 'index')
dd.insert(0, 'test', y_test_data.reset_index().drop(axis = 1, labels = 'index'))
dd.sort_values(by=['act'], inplace=True)

In [ ]:
df_path = os.path.join(PATH, './data/defect_df.p')
dataframe = pd.read_pickle(df_path)
dataframe['defect_formation_energy'] = dataframe['defect_formation_energy'].round(6)
extract_df = dataframe[['name', 'defect_index', 'formula', 'mp_id', 'defect_formation_energy']]
extract_df.columns = ['name', 'defect_index', 'formula', 'mp_id', 'target']

In [ ]:
def get_plot_over_under_estimate(extract_df, dd, threshold=5):
    underestimate = extract_df[extract_df['target'].isin(np.array(dd.iloc[:threshold]['act']).round(6))]
    overesimate = extract_df[extract_df['target'].isin(np.array(dd.iloc[-threshold:]['act']).round(6))]
    
    plt.figure(figsize=(9,6))
    plt.grid(False)
    plt.plot(dd['act'][threshold:-threshold], dd['target'][threshold:-threshold], 'o', ms=9, mec='k', mfc='silver', alpha=0.4)
    plt.plot(dd['act'][:threshold], dd['target'][:threshold], 'o', ms=9, mec='k', mfc='orange', alpha=0.4)
    plt.plot(dd['act'][-threshold:], dd['target'][-threshold:], 'o', ms=9, mec='k', mfc='orange', alpha=0.4)
    plt.ylabel(f'Energy difference (ev)')
    plt.xlabel(f'Oxygen vacancy formation energy (eV)')
    
    top_under = extract_df[extract_df['target'].isin(np.array(dd.iloc[0]['act']).round(6))]['formula']
    top_over = extract_df[extract_df['target'].isin(np.array(dd.iloc[-1]['act']).round(6))]['formula']
    
    plt.annotate(plotter.chemeq(top_under), (dd['act'][:threshold].iloc[0], dd['target'][:threshold].iloc[0]))
    plt.annotate(plotter.chemeq(top_over), (dd['act'][-threshold:].iloc[-1], dd['target'][-threshold:].iloc[-1]))
    plt.axhline(0, color='black')
    return underestimate, overesimate

In [ ]:
underestimate, overesimate = get_plot_over_under_estimate(extract_df, dd, threshold=5)

In [ ]:
#SHAP plot

m = df_classics['model'].iloc[0]
explainer = shap.TreeExplainer(m)
X = X_imp_based_delta
plt.grid(False)
shap_values = explainer.shap_values(X)
i = 4
shap.force_plot(explainer.expected_value, shap_values[i], features=X.iloc[i], feature_names=X.columns)
plt.figure(figsize=(10,8))
plt.grid(False)
shap.summary_plot(shap_values, features=X, feature_names=X.columns, max_display=7, show=False)
plt.gcf().axes[-1].set_aspect(100)
plt.gcf().axes[-1].set_box_aspect(100)
plt.figure(figsize=(10,8))
plt.grid(False)
shap.summary_plot(shap_values, features=X, feature_names=X.columns, plot_type='bar', max_display=7, color='grey')

In [ ]:
#Tree visualization

from sklearn.tree import export_graphviz
#42th tree
sub_tree_42 = df_classics['model'].iloc[1].estimators_[42, 0]

#Export as dot file
export_graphviz(sub_tree_42, out_file='tree.dot', feature_names = X_imp_based_delta.columns, 
                rounded = True, proportion = True, precision = 3, filled = True)

#Convert to png using system command (requires Graphviz)
from subprocess import call 
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

#Display in jupyter notebook
from IPython.display import Image 
Image(filename = 'tree.png')

In [ ]:
for row in range(table.shape[0]):
    act = table['y_test'][row]
    pred = table['predict'][row]
    feature_name = table['feature'][row]
    model = table['model_name'][row]
    plotter.plot_pred_act(act, pred, feature_name, model, reg_line=True, label='')